In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import numpy as np
import warnings
import io
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, MobileNetV3Large 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
from keras.models import load_model
from keras.preprocessing import image

from keras.models import Sequential


In [ ]:
# Data augmentation for training and normalization for both training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for validation data

In [ ]:
# Load the dataset for training and validation
data_dir = 'C:/Users/kunja\Projects/Auto-Segregating-Bin/Tensor-Flow/data'

train_generator = train_datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    os.path.join(data_dir, 'val'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
#x = Dense(128, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks for learning rate scheduling and early stopping
lr_scheduler = LearningRateScheduler(lambda epoch, lr: lr * 0.9 if epoch >= 10 else lr)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit_generator(train_generator,  
                              validation_data=validation_generator,  
                              epochs=10, 
                              verbose=1, 
                              callbacks=[lr_scheduler, early_stopping])

# Save the trained model
model.save('Modelv2_04182024.h5')


In [ ]:
import matplotlib.pyplot as plt

# Extract data from history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(train_loss) + 1)

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'bo-', label='Training loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, 'bo-', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'ro-', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Load the model Modelv3_04172024.h5
#model = tf.keras.models.load_model('Model_03212024_wweights.h5')
model = tf.keras.models.load_model('Modelv2_04182024.h5')
data_dir = "C:/Users/kunja\Projects/Auto-Segregating-Bin/Tensor-Flow/data"
val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = val_datagen.flow_from_directory(
    os.path.join(data_dir, 'val'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

#summ= model.summary()

# Evaluate the model on validation data
print("Evaluating model on validation data...")
val_loss, val_accuracy = model.evaluate(validation_generator)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)
#print(summ)

# predictions for validation data
print("Getting predictions for validation data...")
val_predictions = model.predict(validation_generator)
val_pred_labels = np.argmax(val_predictions, axis=1)
val_true_labels = validation_generator.classes

# classification report
print("Classification Report:")
print(classification_report(val_true_labels, val_pred_labels))

# confusion matrix
print("Confusion Matrix:")
#print(confusion_matrix(val_true_labels, val_pred_labels))
cm = confusion_matrix(val_true_labels, val_pred_labels)
print(cm)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

print("0: 'Cardboard', 1: 'Food Organics', 2: 'glass', 3: 'metal', 4: 'paper', 5: 'plastic', 6: 'Vegetation'")
#print("0:'Non-Biodegradable','1:'Biodegradable'")

#{0: 'Cardboard', 1: 'Food Organics', 2: 'glass', 3: 'metal', 4: 'paper', 5: 'plastic', 6: 'Vegetation'}


In [ ]:
class Predict:

    def __init__(self):
        self.model = load_model('Modelv2_04182024.h5')
        #self.model = load_model('model.tflite')
        self.tsr = image.ImageDataGenerator(rescale=1./255)
        self.class_labels = {0: 'Cardboard', 1: 'Food Organics', 2: 'glass', 3: 'metal', 4: 'paper', 5: 'plastic', 6: 'Vegetation'}
        #self.class_labels = {0: "Non-Biodegradable", 1:"Biodegradable"}

    def inference(self, frame):
        if isinstance(frame, str):
            # If frame is a file path, load the image directly
            img = image.load_img(frame, target_size=(224, 224))
        elif isinstance(frame, np.ndarray):
            # If frame is a numpy array, convert it to bytes and load using BytesIO
            img_bytes = io.BytesIO()
            Image.fromarray(frame).save(img_bytes, format='JPEG')
            img_bytes.seek(0)
            img = image.load_img(img_bytes, target_size=(224, 224))
        else:
            raise ValueError("Unsupported input type for 'frame'")

        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = self.tsr.flow(x, batch_size=1)
        out = self.model.predict(x)
        print(out)
        final = np.argmax(out)
        #print(final)
        label_name = self.class_labels.get(final, 'Unknown')
        print("Predicted class label:", label_name)  # Print the predicted class label
        #print("Predicted class label:", self.new_labels[0] if final in {0, 2, 3, 4, 5} else self.new_labels[1])
        print("Predicted class index:", final)
        with open("transfer.txt", "w") as f:
            f.write(str(final))
        
        

predictor=Predict()
predictor.inference('Food Organics_334.jpg')

In [ ]:
import cv2
import os
import time 

cam = cv2.VideoCapture(0)
cv2.namedWindow("test")
predictor = Predict()

# Creating a directory to store the captured images
output_dir = 'captured_images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
image_count = 0
while image_count < 20:
    check, frame = cam.read()
    if not check:
        print("Failed to capture image")
        break
    cv2.imshow("test", frame)
    predictor.inference(frame)
    #predictor.get_most_frequent_outcome(frames)

    # Save the captured image
    image_count += 1
    image_filename = os.path.join(output_dir, f'image_{image_count}.jpg')
    cv2.imwrite(image_filename, frame)
    time.sleep(1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
cam.release()
cv2.destroyAllWindows()